# 2D Magnetodynamics (Time-Harmonic) - Distribution Transformer

In [1]:
include(joinpath(dirname(@__DIR__), "config.jl"))
paths = get_project_paths("examples_2d")

# Ensure the module is reloaded if changed
if isdefined(Main, :MagnetostaticsFEM)
    println("Reloading MagnetostaticsFEM...")
    # A simple way to force reload in interactive sessions
    try; delete!(LOAD_PATH, paths["SRC_DIR"]); catch; end
    try; delete!(Base.loaded_modules, Base.PkgId(Base.UUID("f8a2b3c4-d5e6-f7a8-b9c0-d1e2f3a4b5c6"), "MagnetostaticsFEM")); catch; end
end
include(joinpath(paths["SRC_DIR"], "MagnetostaticsFEM.jl"))

using LinearAlgebra
using Plots
using LaTeXStrings
using Gmsh: gmsh
using Gridap
using GridapGmsh: GmshDiscreteModel
using .MagnetostaticsFEM
using Printf # For animation title formatting

[ Info: Precompiling Gridap [56d4f2e9-7ea1-5844-9cf6-b9c51ca7ce8e] (cache misses: wrong dep version loaded (2))
ERROR: LoadError: too many parameters for type AbstractTriangular
Stacktrace:
 [1] top-level scope
   @ none:1
 [2] eval(m::Module, e::Any)
   @ Core .\boot.jl:430
 [3] top-level scope
   @ C:\Users\30698\.julia\packages\BlockArrays\TOpxG\src\abstractblockarray.jl:187
 [4] include(mod::Module, _path::String)
   @ Base .\Base.jl:557
 [5] include(x::String)
   @ BlockArrays C:\Users\30698\.julia\packages\BlockArrays\TOpxG\src\BlockArrays.jl:1
 [6] top-level scope
   @ C:\Users\30698\.julia\packages\BlockArrays\TOpxG\src\BlockArrays.jl:49
 [7] include
   @ .\Base.jl:557 [inlined]
 [8] include_package_for_output(pkg::Base.PkgId, input::String, depot_path::Vector{String}, dl_load_path::Vector{String}, load_path::Vector{String}, concrete_deps::Vector{Pair{Base.PkgId, UInt128}}, source::String)
   @ Base .\loading.jl:2881
 [9] top-level scope
   @ stdin:6
in expression starting at C

LoadError: LoadError: Failed to precompile Gridap [56d4f2e9-7ea1-5844-9cf6-b9c51ca7ce8e] to "C:\\Users\\30698\\.julia\\compiled\\v1.11\\Gridap\\jl_BF6C.tmp".
in expression starting at C:\Users\30698\Desktop\tu delft\spring sem\advanced model\FutureDistributionSystemsAM\src\MagnetostaticsFEM.jl:1

## Define Parameters and Paths

In [2]:
# Model Parameters
J0 = 2.2e4       # Source current density [A/m²]
μ0 = 4e-7 * pi  # Vacuum permeability [H/m]
μr_core = 1500.0 # Relative permeability of the core (linear case)
σ_core = 1e6    # Conductivity of the core [S/m] (Laminated)
freq = 50.0     # Frequency [Hz]
ω = 2 * pi * freq # Angular frequency [rad/s]

# FEM Parameters
order = 2
field_type = ComplexF64 # Use ComplexF64 marker for setup_fe_spacesju
dirichlet_tag_name = "Enclosure" # Name of the physical group for Dirichlet BC
dirichlet_value = 0.0 + 0.0im # Dirichlet BC for Az = u + iv

# Paths
mesh_file = joinpath(paths["GEO_DIR"], "2D_quad_transformer.msh")
output_file_base = joinpath(paths["OUTPUT_DIR"], "harmonic")

println("Mesh file: ", mesh_file)
println("Output directory: ", paths["OUTPUT_DIR"])

Mesh file: C:\Users\30698\Desktop\tu delft\spring sem\advanced model\FutureDistributionSystemsAM\examples_2d\geo\2D_quad_transformer.msh

LoadError: InterruptException:

## Setup FEM Problem (Linear Magnetodynamics)

In [3]:
# Load mesh and tags
model, labels, tags = load_mesh_and_tags(mesh_file)

# Get material tags dictionary using the 2D specific function
material_tags = get_material_tags(labels)
println("Material Tags: ", material_tags)

# Set up triangulation and measures
Ω = Triangulation(model)
dΩ = Measure(Ω, 2*order)

reluctivity_func = define_reluctivity(material_tags, μ0, μr_core; core_tag_name="Core")
conductivity_func = define_conductivity(material_tags, σ_core; core_tag_name="Core")
source_current_func = define_current_density(material_tags, J0)

# Setup FE spaces (multi-field: Real, Imag parts)
U, V = setup_fe_spaces(model, order, field_type, dirichlet_tag_name, dirichlet_value)

# Define the weak form problem for the coupled system using the real reference source
problem = magnetodynamics_harmonic_coupled_weak_form(Ω, dΩ, tags, reluctivity_func, conductivity_func, source_current_func, ω)

LoadError: UndefVarError: `load_mesh_and_tags` not defined in `Main`
Suggestion: check for spelling errors or missing imports.

## Solve FEM Problem

In [ ]:
# Solve the real coupled linear FE system
uv = solve_fem_problem(problem, U, V) # uv is a MultiFieldFEFunction

# Extract real and imaginary parts
u = uv[1] # Real part of Az
v = uv[2] # Imag part of Az

LoadError: UndefVarError: `solve_fem_problem` not defined in `Main`
Suggestion: check for spelling errors or missing imports.

## Post-processing

In [ ]:
# Compute B-field (Real and Imag parts)
B_re, B_im = calculate_b_field(uv)

# Compute Eddy Currents (Real and Imag parts)
J_eddy_re, J_eddy_im = calculate_eddy_current(uv, conductivity_func, ω, Ω, tags)

# Define helper functions for magnitude squared
mag_sq_scalar(re, im) = re*re + im*im
mag_sq_vector(re, im) = inner(re, re) + inner(im, im)

# Calculate Magnitudes for saving/plotting using composition
Az_mag = sqrt ∘ (mag_sq_scalar ∘ (u, v))
B_mag = sqrt ∘ (mag_sq_vector ∘ (B_re, B_im))
Jeddy_mag = sqrt ∘ (mag_sq_scalar ∘ (J_eddy_re, J_eddy_im))

# Calculate total current density magnitude (approximation)
# J_total_re = source_current_func_real(tags) + J_eddy_re # Need CellField source
# J_total_im = J_eddy_im
# J_total_mag = sqrt ∘ (mag_sq_scalar ∘ (J_total_re, J_total_im))
# TODO: Need a better way to handle source current in post-processing

# Save results to VTK format
save_results_vtk(Ω, output_file_base, 
    Dict(
        "Az_re" => u, "Az_im" => v, "Az_mag" => Az_mag,
        "B_re" => B_re, "B_im" => B_im, "B_mag" => B_mag,
        "Jeddy_re" => J_eddy_re, "Jeddy_im" => J_eddy_im, "Jeddy_mag" => Jeddy_mag
        # "Jtotal_mag" => J_total_mag
    ),
    save_time_series=true, ω=ω, nframes=100,
)

LoadError: UndefVarError: `calculate_b_field` not defined in `Main`
Suggestion: check for spelling errors or missing imports.

## Visualization (Contour Plots)

In [ ]:
# Plot the magnitude of the magnetic vector potential
plot_contour_2d(Ω, Az_mag, title="Magnetic Vector Potential Magnitude |Az|", output_path=joinpath(paths["OUTPUT_DIR"], "Az_magnitude_contour.pdf"))

# Plot the magnitude of the magnetic flux density
plot_contour_2d(Ω, B_mag, title="Magnetic Flux Density Magnitude |B|", output_path=joinpath(paths["OUTPUT_DIR"], "B_magnitude_contour.pdf"))

# Plot the magnitude of the eddy current density
plot_contour_2d(Ω, Jeddy_mag, title="Eddy Current Density Magnitude |Jeddy|", output_path=joinpath(paths["OUTPUT_DIR"], "Jeddy_magnitude_contour.pdf"))

LoadError: UndefVarError: `plot_contour_2d` not defined in `Main`
Suggestion: check for spelling errors or missing imports.